Mini-projet XAI — **ProtoPNet (Prototypical Part Network)**
"This Looks Like That: Deep Learning for Interpretable Image Recognition"

**Étudiant**: CHAIBOU SAIDOU Abdoulaye<br>
**Professeur**: Tawfik Masrour<br>
**Date**: 16 Décembre 2025<br>
**Filière**: IATD-SI<br>
**Etablissement**: [ENSAM-Meknès](https://ensam.umi.ac.ma)

**Remerciements**:
Ce projet est basé sur l'article "This Looks Like That: Deep Learning for 
Interpretable Image Recognition" (NeurIPS 2019), par Chaofan Chen*, Oscar Li*, 
Chaofan Tao, Alina Jade Barnett, Jonathan Su, et Cynthia Rudin (Duke University).
Code original: [PPNet repo](https://github.com/cfchen-duke/ProtoPNet).

**Infos**: Ce notebook présente brièvement la technique interprétable d'explication des réseaux de convolutifs appelée Prototypical Part Network par ses auteurs officiels, sous forme de mini-projet (IA Explicable) de classe. Nous utilisons egalement un modèle pré-entraîné par *Alina Jade Barnett* pour l'analyse de la methode, par soucis de simplicité et de gain de temps.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from PIL import Image
import cv2
import os
from pathlib import Path
import copy

# Configuration matplotlib pour de belles visualisations
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

# Force CPU pour stabilité (recommandé pour ce projet)
device = torch.device("cpu")
print(f"Device utilisé: {device} (mode stable)")

PyTorch version: 2.5.0+cpu
CUDA disponible: False
Device utilisé: cpu (mode stable)


🔴 LE PROBLÈME DES BOÎTES NOIRES :

Les réseaux de neurones profonds (CNN) sont extrêmement performants pour la
classification d'images, MAIS ils sont des "boîtes noires" inexplicables :

  Médecin : "Pourquoi le modèle dit que c'est un cancer ?"
  CNN Classique : "🤷 Résultat = 95% cancer"
  
  Juge : "Sur quelle base le modèle a identifié cette personne ?"
  CNN Classique : "🤷 Confiance = 98%"

❌ Conséquences :
  - Impossible de vérifier le raisonnement
  - Risque de biais cachés
  - Manque de confiance des experts
  - Non-conforme aux régulations (RGPD, FDA)

💡 LA SOLUTION PROTOPNET :

Au lieu d'une boîte noire, ProtoPNet fonctionne comme l'esprit humain :
raisonnement par SIMILITUDE avec des CAS MÉMORISÉS (prototypes).

  Médecin : "Pourquoi cancer ?"
  ProtoPNet : "Cette région ressemble à 85% à ce prototype de mélanome,
               cette autre région ressemble à 78% à cet autre prototype..."
  Médecin : ✅ "Je peux vérifier ! Le raisonnement est correct."


## TYPE D'EXPLICATION

📊 GLOBALE + LOCALE :
  - GLOBALE : Les prototypes sont appris une fois pour toutes les classes
  - LOCALE : Chaque prédiction s'explique par similarité aux prototypes

🖼️ DOMAINE D'APPLICATION : IMAGES / CNN
  - Classification d'oiseaux (dataset CUB-200)
  - Diagnostic médical (dermatologie, radiologie)
  - Contrôle qualité industriel
  - Toute tâche de classification fine-grained

🎯 AVANTAGE CLÉ : INTERPRÉTABILITÉ INTRINSÈQUE

Contrairement à LIME, SHAP, Grad-CAM (méthodes POST-HOC qui approximent),
ProtoPNet est INTRINSÈQUEMENT INTERPRÉTABLE :
  - L'explication fait partie de l'architecture
  - Pas d'approximation
  - Fidélité garantie à 100%

## ANALOGIE HUMAINE

Imaginez que vous apprenez à reconnaître des oiseaux :

1️⃣ PHASE D'APPRENTISSAGE (mémorisation de prototypes) :
   
   Vous voyez beaucoup d'oiseaux et mémorisez des CARACTÉRISTIQUES TYPIQUES :
   ```
   🐦 Cardinals :
      → Prototype A : "crête rouge vif caractéristique"
      → Prototype B : "bec orange pointu"
      → Prototype C : "masque noir autour des yeux"
   
   🐦 Geais bleus :
      → Prototype D : "plumes bleues brillantes"
      → Prototype E : "bande noire sur la tête"
      → Prototype F : "bout des ailes blanc"
```
2️⃣ PHASE DE RECONNAISSANCE (nouvelle image) :
   
   Vous voyez un nouvel oiseau inconnu :
   
   
   | 🐦 Oiseau mystère |
|------------------|
| ✓ Cette partie ressemble beaucoup au **Prototype A** |
| ✓ Cette autre partie ressemble au **Prototype B** |
| ✗ Pas de similarité avec les prototypes de **geai bleu** |

   
   CONCLUSION : "C'est probablement un cardinal !"

3️⃣ EXPLICATION :
   
   Si on vous demande "Pourquoi cardinal ?" :
   ```
   → "Parce que sa crête ressemble au Prototype A (85% similaire)"
   → "Et son bec ressemble au Prototype B (78% similaire)"
   ```

## C'EST EXACTEMENT CE QUE FAIT PROTOPNET !

🏗️ ARCHITECTURE EN 3 ÉTAPES :
```
   ┌─────────────────┐
   │  Image Input    │  📸 Exemple : photo d'oiseau 224×224×3
   │  (224×224×3)    │
   └────────┬────────┘
            │
            ▼
   ┌─────────────────┐
   │  CNN Backbone   │  🔧 VGG16 / ResNet : extraction de features
   │  (ex: VGG16)    │     Sortie : feature map H×W×D
   └────────┬────────┘     (ex: 14×14×128)
            │
            ▼
   ┌─────────────────┐
   │ Prototype Layer │  🎯 Compare chaque région aux m prototypes
   │  (m prototypes) │     Calcule similarités spatiales
   └────────┬────────┘     Sortie : m scores de similarité
            │
            ▼
   ┌─────────────────┐
   │Classification   │  📊 Combine les similarités avec poids appris
   │     Layer       │     Sortie : scores par classe
   └─────────────────┘
```
![diagram of the architecture](images/architectureppnet.png)
📤 QUE PRODUIT LA MÉTHODE ?

Pour chaque image à classifier :

1. HEATMAP DE SIMILARITÉ:
```
 pour chaque prototype activé → Montre QUELLE RÉGION de l'image ressemble au prototype
 ```

2. TOP-K PROTOTYPES:
```
les plus similaires → "Les 5 prototypes qui matchent le mieux avec cette image"
```
3. EXPLICATION VISUELLE :
   → "Cette partie [bbox rouge] ressemble à 85% au Prototype #42"
   → "Prototype #42 = crête rouge d'un cardinal"

4. RAISONNEMENT COMPLET :
```
   → Similarité prototype 1 : 0.85  ×  Poids 1.2  =  +1.02
   → Similarité prototype 2 : 0.78  ×  Poids 0.9  =  +0.70
   → Similarité prototype 3 : 0.12  ×  Poids -0.3 =  -0.04
   → ──────────────────────────────────────────────────
   → TOTAL score pour "Cardinal" : 1.68 → Softmax → 92%
```
🎯 FAMILLE XAI :
Comparaison entre ProtoPNet (interprétabilité intrinsèque) et les méthodes post-hoc courantes (LIME, SHAP, Grad-CAM) :

| Aspect | ProtoPNet (interprétabilité intrinsèque) | Méthodes post-hoc (LIME, SHAP, Grad-CAM) |
|------|------------------------------------------|------------------------------------------|
| Moment de l’explication | **Pendant la prédiction** | Après la prédiction |
| Principe | Raisonnement par **similarité avec des prototypes** | Approximation ou analyse a posteriori |
| Lien avec le modèle | L’explication **fait partie du modèle** | L’explication est **ajoutée par-dessus** |
| Fidélité à la décision | **100 % fidèle** (pas d’approximation) | Approximative |
| Type d’explication | « *Cette partie ressemble à ce prototype* » | Importance de pixels / features |
| Raisonnement | **Case-Based Reasoning** (par cas) | Attribution de contribution |
| Stabilité | Stable (mêmes prototypes) | Peut varier selon les paramètres |
| Coût de calcul | Modéré | SHAP souvent **très coûteux** |
| Compréhensible par humain | **Oui (visuel + sémantique)** | Parfois difficile à interpréter |
| Confiance utilisateur | Élevée | Variable |


### NOTATIONS

- $x ∈ ℝ^{224\times224\times3}$     : image d'entrée RGB
- $f(x) ∈ ℝ^{H\times\text{W}\times\text{D}}$       : carte de features extraite par le CNN
                          (ex: H=14, W=14, D=128 pour input $224\times224$)
- $P = {p₁, ..., pₘ}$     : ensemble des m prototypes
                          (chaque $p_j ∈ ℝ^D$, même dimension que features)
- $d(·,·)$                : distance L2 (euclidienne)
- $K$                     : nombre de classes

### ÉQUATION 1 :
#### SIMILARITÉ AUX PROTOTYPES

$$
\textbf{Pour un prototype } p_j \textbf{ donné, on calcule sa similarité avec l'image } x :
$$

$$
\mathrm{sim}(x, p_j)
= \max_{h,w} \; \exp\!\left(- d^2\!\left(f(x)_{h,w},\, p_j\right)\right)
$$

$$
\textbf{Décomposition :}
$$

$$
\textbf{1. } f(x)_{h,w} \in \mathbb{R}^D
\quad \text{: vecteur de caractéristiques à la position spatiale } (h,w)
$$

$$
\textbf{2. } d^2\!\left(f(x)_{h,w}, p_j\right)
= \left\| f(x)_{h,w} - p_j \right\|^2
= \sum_{k=1}^{D} \left( f(x)_{h,w,k} - p_j[k] \right)^2
$$

$$
\textbf{3. } \exp(-d^2)
\quad \text{: conversion distance } \rightarrow \text{ similarité}
$$

$$
\begin{aligned}
d^2 &= 0 \quad &\Rightarrow \quad \exp(0) &= 1.0 \quad &(\text{identique}) \\
d^2 &= 1 \quad &\Rightarrow \quad \exp(-1) &= 0.37 \quad &(\text{assez similaire}) \\
d^2 &= 5 \quad &\Rightarrow \quad \exp(-5) &= 0.007 \quad &(\text{très différent})
\end{aligned}
$$

$$
\textbf{4. } \max_{h,w}
\quad \text{: recherche de la meilleure correspondance spatiale}
$$

$$
\Rightarrow \text{on sélectionne la région de l'image la plus proche du prototype}
$$

$$
\textbf{Intuition :}
\quad
\text{``Quelle est la région de l'image qui ressemble le plus au prototype } p_j \text{ ?''}
$$



#### PRÉDICTION FINALE

##### Score de classe (logit)

Le score ($logit$) pour la classe $k$ est :

$$
g_k(x) = \sum_{j} w_{k,j} \cdot \mathrm{sim}(x, p_j)
$$

##### Décomposition

- $$ w_{k,j} \in \mathbb{R} $$
  poids appris représentant l'importance du prototype $p_j$ pour la classe $k$

- **Somme pondérée**  
  on combine toutes les similarités entre l'image et les prototypes

- **Transformation en probabilités (softmax)** :

$$
P(\text{classe} = k \mid x)
=
\frac{\exp\left(g_k(x)\right)}
{\sum_{k'} \exp\left(g_{k'}(x)\right)}
$$

---

##### 💡 Intuition

> *« Chaque prototype **vote** pour les classes en fonction de sa similarité avec l'image »*

---

##### Exemple concret

###### Classe **"Cardinal"** ($ k = 5 $)

- **Prototype 25** (crête rouge)  
  $$ \mathrm{sim} = 0.85, \quad w_{5,25} = 1.2 \quad \Rightarrow \quad +1.02 $$

- **Prototype 26** (bec orange)  
  $$ \mathrm{sim} = 0.78, \quad w_{5,26} = 0.9 \quad \Rightarrow \quad +0.70 $$

- **Prototype 27** (masque noir)  
  $$ \mathrm{sim} = 0.65, \quad w_{5,27} = 0.7 \quad \Rightarrow \quad +0.46 $$

- **Autres 97 prototypes**  
  contribution faible  
  $$ \Rightarrow +0.32 $$

---

###### Score total

$$
g_5(x) = 2.50
\quad \Rightarrow \quad
P(\text{Cardinal} \mid x) = 0.92
$$


#### FONCTION DE PERTE (ENTRAÎNEMENT)
L'entraînement optimise une fonction de perte multi-objectifs :
$$
    L_{total} = L_{cls} + λ₁ L_{clst} + λ₂ L_{sep} + λ₃ L_{reg}
$$
Décomposition :

1️⃣ $L_{cls}$ = $\text{CrossEntropy}(g(x), y_{true})$
   → Perte de classification standard<br>
   → Force le modèle à prédire correctement

2️⃣ $L_{clst}$ = $- \sum_{j} \min_{x \in \text{classe}(p_j)}d^2\!\left(f(x),\, p_j\right)$
   → "Clustering loss"<br>
   → Force chaque prototype à être PROCHE d'au moins un exemple 
     de SA PROPRE classe<br>
   → Garantit que les prototypes sont représentatifs

3️⃣ $L_{sep}$ = $\sum_{j} \min_{x \notin \text{classe}(p_j)}\left( - d^2\!\left(f(x),\, p_j\right) \right)$
   → "Separation loss"<br>
   → Force chaque prototype à être ÉLOIGNÉ des exemples 
     des AUTRES classes<br>
   → Garantit la discriminativité

4️⃣ $L_{reg}$ = $\| W \|_1$
   → Régularisation L1 sur les poids<br>
   → Encourage la sparsité : peu de prototypes par classe<br>
   → Facilite l'interprétation<br>

💡 INTUITION :
"Les prototypes doivent être proches de leur classe, éloignés des autres,
 et le modèle doit bien classifier avec peu de prototypes actifs"


### LIEN AVEC VOS CONNAISSANCES

🔗 CNN / Rétropropagation :
   - $f(x)$ = forward pass standard d'un CNN (VGG16, ResNet)
   - Les gradients se propagent à travers les prototypes
   - Même mécanisme d'optimisation (Adam, SGD)

🔗 Max Pooling :
   - $max_{h,w}$ dans $\text{sim}(x, p_j)$ = max pooling spatial guidé
   - Au lieu de max(features), on fait max(similarités)

🔗 Noyau RBF (Radial Basis Function) :
   - $exp(-d²)$ = noyau gaussien, classique en ML
   - Utilisé en SVM, clustering, réseaux de neurones

🔗 K-means / Clustering :
   - Les prototypes jouent le rôle de "centres de clusters"
   - $L_{clst}$ ≈ minimiser distance intra-classe (K-means)
   - $L_{sep}$ ≈ maximiser distance inter-classe

🔗 Attention Mechanism :
   - $max_{h,w}$ = forme d'attention spatiale "hard"
   - Le modèle "attend" aux régions importantes